In [89]:
import pandas as pd
import numpy as np
from pathlib import Path
from thefuzz.process import extractOne
from tqdm import tqdm

tqdm.pandas()

In [90]:
stats = pd.read_csv(Path.cwd()/'prepped/player_stats.csv', index_col=False)
transfers = pd.read_csv(Path.cwd()/'prepped/data.csv', index_col=False).drop(columns=['position'])
elo = pd.read_csv(Path.cwd()/'prepped/elo.csv', index_col=False).drop(columns=['Unnamed: 0'])
display(stats, transfers, elo)

/var/folders/jm/4ddznsn94j7_pznhvk_1rwlw0000gn/T/ipykernel_65290/2189893827.py:1: DtypeWarning: Columns (4,6,15,16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  stats = pd.read_csv(Path.cwd()/'prepped/player_stats.csv', index_col=False)


,playerid,player,league,team,pos_23/24,marketval_23/24,season_23/24,matchesplayed_23/24,minsplayed_23/24,goals_23/24,...,prevgoals90_18/19,bpassesrcvd90_18/19,exits90_18/19,gkaerduels90_18/19,fk90_18/19,directfk90_18/19,directfkontrgtpct_18/19,corners90_18/19,pens_18/19,pencnvrspct_18/19
0,246963.0,C. Morris,GB1,Luton Town,CF,13000000.0,23/24,36.0,2987.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,395516.0,M. Diaby,GB1,Aston Villa,"CF, AMF",60000000.0,23/24,36.0,2249.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,525247.0,M. Kilman,GB1,Wolverhampton Wanderers,"RCB, LCB, CB",35000000.0,23/24,36.0,3687.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,495666.0,W. Saliba,GB1,Arsenal,RCB,75000000.0,23/24,36.0,3656.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,258889.0,D. Solanke,GB1,Bournemouth,CF,25000000.0,23/24,36.0,3584.0,18.0,...,NaN,0.00,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18132,14977.0,C. Janker,L1,Augsburg,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
18133,55114.0,O. Sorg,L1,Hannover 96,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.84,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
18134,53408.0,F. Giefer,L1,Augsburg,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.131,0.00,1.40,0.93,NaN,NaN,NaN,0.0,0.0,0.0
18135,12864.0,J. Drobný,L1,Fortuna Düsseldorf,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.308,0.00,2.37,1.42,NaN,NaN,NaN,0.0,0.0,0.0


,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan
0,Neymar,25,2017,100.00,Spain,LaLiga,FC Barcelona,France,Ligue 1,Paris Saint-Germain,summer,222.00,False
1,Ousmane Dembélé,20,2017,33.00,Germany,Bundesliga,Borussia Dortmund,Spain,LaLiga,FC Barcelona,summer,135.00,False
2,Romelu Lukaku,24,2017,50.00,England,Premier League,Everton FC,England,Premier League,Manchester United,summer,84.70,False
3,Álvaro Morata,24,2017,40.00,Spain,LaLiga,Real Madrid,England,Premier League,Chelsea FC,summer,66.00,False
4,Benjamin Mendy,23,2017,13.00,Monaco,Ligue 1,AS Monaco,England,Premier League,Manchester City,summer,57.50,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27982,Juan Mercado,26,2023,0.60,Bolivia,División Profesional Clausura,Club Deportivo Oriente Petrolero,Bolivia,División Profesional Clausura,Universitario de Vinto,winter,0.00,False
27983,Horacio Torres,24,2023,0.60,Mexico,Liga Expansión MX Cl.,Mineros de Zacatecas,Mexico,Liga Expansión MX Cl.,Atlético La Paz,winter,0.00,False
27984,Joe Gauci,23,2023,0.55,Australia,A-League Men,Adelaide United,England,Premier League,Aston Villa,winter,1.50,False
27985,Shihao Wei,28,2023,0.55,China,Super League,Wuhan Three Towns,China,Super League,Chengdu Rongcheng,winter,0.64,False


,season,league,Club,Elo,avg_elo
0,17/18,GB1,Chelsea,1909,1651.333333
1,17/18,GB1,Tottenham,1887,1651.333333
2,17/18,GB1,Man City,1866,1651.333333
3,17/18,GB1,Man United,1857,1651.333333
4,17/18,GB1,Arsenal,1848,1651.333333
...,...,...,...,...,...
2054,23/24,SC1,Kilmarnock,1236,1319.923077
2055,23/24,SC1,St Johnstone,1235,1319.923077
2056,23/24,SC1,Ross County,1221,1319.923077
2057,23/24,SC1,Dundee United,1198,1319.923077


In [91]:
league_map = {
    'LaLiga': 'SP1',
    'Bundesliga': 'L1',
    'Premier League': 'EN1',
    'Ligue 1': 'FR1',
    'Serie A': 'IT1',
    'Eredivisie': 'NL1',
    'Liga Portugal': 'PO1',
    'Liga NOS': 'PO1',
    'Jupiler Pro League': 'BE1',
    'JPL': 'BE1',
    'Superliga': 'DK1',
    'Super League': 'GR1',
    'Premier Liga': 'RU1',
    'Premiership': 'SC1',
    'Süper Lig': 'TR1'
}


In [92]:
transfers.league_from = transfers.league_from.replace(league_map)
transfers.league_to = transfers.league_to.replace(league_map)

In [93]:
transfers.iloc[14]

name            Corentin Tolisso
age                           22
season                      2017
market_value                22.0
country_from              France
league_from                  FR1
club_from         Olympique Lyon
country_to               Germany
league_to                     L1
club_to            Bayern Munich
window                    summer
fee                         41.5
loan                       False
Name: 14, dtype: object

In [94]:
def short_name_gen(name):
    name = name.split(' ')
    if len(name) == 1: 
        return name[0]
    name[0] = name[0][0] + '.'
    return ' '.join(name)

def match_wyscout_to_tm(row: pd.Series, thresh_player=50, thres_team=0): 
    short_name = short_name_gen(row['name'])
    
    club_from_wyscout = extractOne(
        query=row.club_from, score_cutoff=thres_team,
        choices=stats[stats['league'] == row.league_from]['team'].unique().tolist())[0]
    
    club_to_wyscout = extractOne(
        query=row.club_to, score_cutoff=thres_team,
        choices=stats[stats['league'] == row.league_to]['team'].unique().tolist())[0]
    
    name_wyscout = extractOne(
        query=row['name'], score_cutoff=thresh_player,
        choices=stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout)]['player'].unique().tolist()
        ) or (None, 0)
    
    short_name_wyscout = extractOne(
        query=short_name, score_cutoff=thresh_player,
        choices=stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout)]['player'].unique().tolist()
        ) or (None, 0)
    
    if club_from_wyscout == None or club_to_wyscout == None or (name_wyscout[0] == None and short_name_wyscout[0] == None):
        row['name'] = None
        row['playerid'] = None
        return row
    else:
        row['name'] = max([name_wyscout, short_name_wyscout], key = lambda t: t[1])[0]
        if stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout) & (stats['player'] == row['name'])]['playerid'].size > 1:
            display(row.club_to, club_to_wyscout, row['name'], short_name, name_wyscout, short_name_wyscout)
            display(stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout) & (stats['player'] == row['name'])])
            row['playerid'] = np.nan
            return row
        row['playerid'] = stats[(stats[f'league'] == row.league_to) & (stats['team'] == club_to_wyscout) & (stats['player'] == row['name'])]['playerid'].item()
        row['club_from'] = club_from_wyscout
        row['club_to'] = club_to_wyscout

    return row

def match_elo_to_tm(row, thresh_team=0):
    club_from_elo = extractOne(
        query=row.club_from,
        choices=elo[(elo.league == row.league_from) & (elo.season == row.season)]['Club'].unique().tolist())[0]
    # print(club_from_elo)
    club_to_elo = extractOne(
        query=row.club_to,
        choices=elo[(elo.league == row.league_to) & (elo.season == row.season)]['Club'].unique().tolist())[0]
    # print(club__elo)
    row['club_from_elo'] = elo[(elo.league == row.league_from) & (elo.season == row.season) & (elo.Club == club_from_elo)]['Elo'].item()
    row['club_to_elo'] = elo[(elo.league == row.league_to) & (elo.season == row.season) & (elo.Club == club_to_elo)]['Elo'].item()
    row['league_from_elo'] = elo[(elo.league == row.league_from) & (elo.season == row.season) & (elo.Club == club_from_elo)]['avg_elo'].item()
    row['league_to_elo'] = elo[(elo.league == row.league_to) & (elo.season == row.season) & (elo.Club == club_to_elo)]['avg_elo'].item()
    return row

In [95]:
competitions = ['GB1', 'FR1', 'TR1', 'NL1', 'SC1', 'IT1', 'DK1', 'PO1', 'ES1', 'BE1', 'RU1', 'GR1', 'L1']
countries = ['Spain', 'Germany', 'England', 'France', 'Italy', 'Netherlands', 'Portugal', 'Belgium', 
             'Denmark', 'Greece', 'Russia', 'Scotland', 'Türkiye']
transfers = transfers[transfers.league_to.isin(competitions) &
                      transfers.country_to.isin(countries)]
transfers = transfers[transfers.league_from.isin(competitions) & 
                      transfers.country_from.isin(countries)]
transfers['season'] = transfers['season'].map({2018: '18/19', 2019: '19/20', 2020: '20/21', 2021: '21/22', 2022: '22/23', 2023: '23/24'})
transfers = transfers[transfers.season.notnull()] # remove 17/18 season
# transfers['club_to'] = transfers.club_to.str.replace('Inter Milan', 'Internazionale') \
#                                         .str.replace('Paris Saint-Germain', 'PSG') \
#                                         .str.replace('Espanyol Barcelona', 'Espanyol')
# transfers['club_from'] = transfers.club_from.str.replace('Inter Milan', 'Internazionale') \
#                                         .str.replace('Paris Saint-Germain', 'PSG') \
#                                         .str.replace('Espanyol Barcelona', 'Espanyol')

In [96]:
transfers

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan
4010,Douglas Costa,27,18/19,55.00,Germany,L1,Bayern Munich,Italy,IT1,Juventus FC,summer,40.0,False
4011,Gonzalo Higuaín,30,18/19,55.00,Italy,IT1,Juventus FC,Italy,IT1,AC Milan,summer,10.2,True
4015,Radja Nainggolan,30,18/19,45.00,Italy,IT1,AS Roma,Italy,IT1,Inter Milan,summer,38.0,False
4016,Patrik Schick,22,18/19,20.00,Italy,IT1,UC Sampdoria,Italy,IT1,AS Roma,summer,42.0,False
4022,Leon Goretzka,23,18/19,40.00,Germany,L1,FC Schalke 04,Germany,L1,Bayern Munich,summer,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27535,Nikolai Baden Frederiksen,23,23/24,0.70,Netherlands,NL1,Vitesse Arnhem,Denmark,DK1,Lyngby BK,winter,0.0,False
27695,Sylla Sow,27,23/24,0.65,Netherlands,NL1,Go Ahead Eagles,Netherlands,NL1,NEC Nijmegen,winter,0.0,False
27811,Ertaç Özbir,34,23/24,0.60,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,MKE Ankaragücü,winter,0.0,False
27829,Igor Yurganov,30,23/24,0.60,Russia,RU1,FC Sochi,Russia,RU1,Fakel Voronezh,winter,0.0,False


In [97]:
transfers.iloc[1:2]

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan
4011,Gonzalo Higuaín,30,18/19,55.0,Italy,IT1,Juventus FC,Italy,IT1,AC Milan,summer,10.2,True


In [98]:
transfers = transfers.progress_apply(match_elo_to_tm, axis=1)

100%|██████████| 3663/3663 [00:57<00:00, 63.93it/s]


In [99]:
transfers

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,club_from_elo,club_to_elo,league_from_elo,league_to_elo
4010,Douglas Costa,27,18/19,55.00,Germany,L1,Bayern Munich,Italy,IT1,Juventus FC,summer,40.0,False,1947,1961,1575.566667,1591.500000
4011,Gonzalo Higuaín,30,18/19,55.00,Italy,IT1,Juventus FC,Italy,IT1,AC Milan,summer,10.2,True,1961,1723,1591.500000,1591.500000
4015,Radja Nainggolan,30,18/19,45.00,Italy,IT1,AS Roma,Italy,IT1,Inter Milan,summer,38.0,False,1848,1755,1591.500000,1591.500000
4016,Patrik Schick,22,18/19,20.00,Italy,IT1,UC Sampdoria,Italy,IT1,AS Roma,summer,42.0,False,1603,1848,1591.500000,1591.500000
4022,Leon Goretzka,23,18/19,40.00,Germany,L1,FC Schalke 04,Germany,L1,Bayern Munich,summer,0.0,False,1739,1947,1575.566667,1575.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27535,Nikolai Baden Frederiksen,23,23/24,0.70,Netherlands,NL1,Vitesse Arnhem,Denmark,DK1,Lyngby BK,winter,0.0,False,1501,1327,1490.285714,1426.857143
27695,Sylla Sow,27,23/24,0.65,Netherlands,NL1,Go Ahead Eagles,Netherlands,NL1,NEC Nijmegen,winter,0.0,False,1428,1441,1490.285714,1490.285714
27811,Ertaç Özbir,34,23/24,0.60,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,MKE Ankaragücü,winter,0.0,False,1530,1383,1435.363636,1435.363636
27829,Igor Yurganov,30,23/24,0.60,Russia,RU1,FC Sochi,Russia,RU1,Fakel Voronezh,winter,0.0,False,1443,1414,1446.166667,1446.166667


In [100]:
transfers = transfers.progress_apply(match_wyscout_to_tm, axis=1).dropna(subset=['name'])
transfers

  1%|          | 26/3663 [00:01<02:14, 27.14it/s]

'FC Schalke 04'

'Schalke 04'

'S. Sané'

'S. Sané'

...

('A. Morelos', 50)

('R. McCrorie', 54)

,playerid,player,league,team,pos_23/24,marketval_23/24,season_23/24,matchesplayed_23/24,minsplayed_23/24,goals_23/24,...,prevgoals90_18/19,bpassesrcvd90_18/19,exits90_18/19,gkaerduels90_18/19,fk90_18/19,directfk90_18/19,directfkontrgtpct_18/19,corners90_18/19,pens_18/19,pencnvrspct_18/19
6674,244260.0,R. McCrorie,SC1,Rangers,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7039,293746.0,R. McCrorie,SC1,Rangers,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.51,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


100%|██████████| 3663/3663 [02:39<00:00, 22.98it/s]


,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,club_from_elo,club_to_elo,league_from_elo,league_to_elo,playerid
4010,Douglas Costa,27,18/19,55.00,Germany,L1,Bayern München,Italy,IT1,Juventus,summer,40.0,False,1947,1961,1575.566667,1591.500000,75615.0
4011,G. Bonaventura,30,18/19,55.00,Italy,IT1,Juventus,Italy,IT1,Milan,summer,10.2,True,1961,1723,1591.500000,1591.500000,57280.0
4015,R. Nainggolan,30,18/19,45.00,Italy,IT1,Roma,Italy,IT1,Inter Milan,summer,38.0,False,1848,1755,1591.500000,1591.500000,41982.0
4016,P. Schick,22,18/19,20.00,Italy,IT1,Sampdoria,Italy,IT1,Roma,summer,42.0,False,1603,1848,1591.500000,1591.500000,242086.0
4022,L. Goretzka,23,18/19,40.00,Germany,L1,Schalke 04,Germany,L1,Bayern München,summer,0.0,False,1739,1947,1575.566667,1575.566667,153084.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27535,N. Geertsen,23,23/24,0.70,Netherlands,NL1,Vitesse,Denmark,DK1,Lyngby,winter,0.0,False,1501,1327,1490.285714,1426.857143,157912.0
27695,S. Hansen,27,23/24,0.65,Netherlands,NL1,Go Ahead Eagles,Netherlands,NL1,NEC,winter,0.0,False,1428,1441,1490.285714,1490.285714,485582.0
27811,E. Bekiroğlu,34,23/24,0.60,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,Ankaragücü,winter,0.0,False,1530,1383,1435.363636,1435.363636,327402.0
27829,S. Bryzgalov,30,23/24,0.60,Russia,RU1,Sochi,Russia,RU1,Fakel,winter,0.0,False,1443,1414,1446.166667,1446.166667,137179.0


In [101]:
transfers = transfers.merge(stats, how='left', left_on=['playerid', 'club_from'], right_on=['playerid', 'team'], suffixes=('', '_pre'))
transfers = transfers.merge(stats, how='left', left_on=['playerid', 'club_to'], right_on=['playerid', 'team'], suffixes=('', '_post'))
transfers

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,prevgoals90_18/19_post,bpassesrcvd90_18/19_post,exits90_18/19_post,gkaerduels90_18/19_post,fk90_18/19_post,directfk90_18/19_post,directfkontrgtpct_18/19_post,corners90_18/19_post,pens_18/19_post,pencnvrspct_18/19_post
0,Douglas Costa,27,18/19,55.00,Germany,L1,Bayern München,Italy,IT1,Juventus,...,NaN,0.00,NaN,NaN,0.13,0.0,0.0,1.19,0.0,0.0
1,G. Bonaventura,30,18/19,55.00,Italy,IT1,Juventus,Italy,IT1,Milan,...,NaN,0.13,NaN,NaN,0.00,0.0,0.0,0.27,0.0,0.0
2,R. Nainggolan,30,18/19,45.00,Italy,IT1,Roma,Italy,IT1,Inter Milan,...,NaN,0.39,NaN,NaN,0.00,0.0,0.0,0.00,0.0,0.0
3,P. Schick,22,18/19,20.00,Italy,IT1,Sampdoria,Italy,IT1,Roma,...,NaN,0.00,NaN,NaN,0.00,0.0,0.0,0.00,0.0,0.0
4,L. Goretzka,23,18/19,40.00,Germany,L1,Schalke 04,Germany,L1,Bayern München,...,NaN,0.51,NaN,NaN,0.00,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668,N. Geertsen,23,23/24,0.70,Netherlands,NL1,Vitesse,Denmark,DK1,Lyngby,...,NaN,0.74,NaN,NaN,0.00,0.0,0.0,0.00,0.0,0.0
3669,S. Hansen,27,23/24,0.65,Netherlands,NL1,Go Ahead Eagles,Netherlands,NL1,NEC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3670,E. Bekiroğlu,34,23/24,0.60,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,Ankaragücü,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3671,S. Bryzgalov,30,23/24,0.60,Russia,RU1,Sochi,Russia,RU1,Fakel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
def season_to_offset(row):
    seasons = ['18/19', '19/20', '20/21', '21/22', '22/23', '23/24']
    transfer_season = seasons.index(row['season'])
    cols = row.index.tolist()
    for i in range(len(cols)):
        for s in seasons:
            if s in cols[i]:
                offset = seasons.index(s)
                idx = offset - transfer_season
                # print(cols[i], offset, transfer_season, idx, sep=', ')
                # if np.sign(idx) > 0:
                #     idx += 1
                # elif row['window'] == 'winter':
                #         idx -= 1
                cols[i] = cols[i].replace(f'_{s}','') + f'_{str(idx)}'
                continue
    
    row.index = cols
    row = row[row.index.isin(transfers.columns.to_list()[:17]) |
              ((row.index.str.contains('post_')) & (row.index.to_series().apply(lambda x: int(x[-1]) >= 0 if x[-1].isnumeric() and x[-2] != '-' else False))) | 
              ((~row.index.str.contains('post_')) & (row.index.to_series().apply(lambda x: int(x[-2:]) <= 0 if x[-1].isnumeric() and x[-2] == '-' else False)))]
    return row.to_frame().T

season_to_offset(transfers.iloc[1616,:]).columns.tolist()

['name',
 'age',
 'season',
 'market_value',
 'country_from',
 'league_from',
 'club_from',
 'country_to',
 'league_to',
 'club_to',
 'window',
 'fee',
 'loan',
 'club_from_elo',
 'club_to_elo',
 'league_from_elo',
 'league_to_elo',
 'pos_-1',
 'marketval_-1',
 'season_-1',
 'matchesplayed_-1',
 'minsplayed_-1',
 'goals_-1',
 'xg_-1',
 'assists_-1',
 'xa_-1',
 'duels90_-1',
 'duelswonpct_-1',
 'birthcountry_-1',
 'passpcountry_-1',
 'foot_-1',
 'height_-1',
 'weight_-1',
 'loanflg_-1',
 'succssdefact90_-1',
 'defduels90_-1',
 'defduelswpct_-1',
 'aerduels90_-1',
 'aerduelspct_-1',
 'sltackles90_-1',
 'padjsltackles_-1',
 'shotblocks90_-1',
 'intrcpt90_-1',
 'padjintrcpt_-1',
 'fouls90_-1',
 'ycards_-1',
 'ycards90_-1',
 'rcards_-1',
 'rcards90_-1',
 'sattact90_-1',
 'goals90_-1',
 'npgoals_-1',
 'npgoals90_-1',
 'xg90_-1',
 'hdrgoals_-1',
 'hdrgoals90_-1',
 'shots_-1',
 'shots90_-1',
 'ontrgtpct_-1',
 'goalcnvrsnpct_-1',
 'assists90_-1',
 'crosses90_-1',
 'acccrossespct_-1',
 'crossesl

In [103]:
rows = []
for i, row in tqdm(transfers.iterrows(), total=transfers.shape[0]):
    rows.append(season_to_offset(row))

df = pd.concat(rows, ignore_index=True)
df

100%|██████████| 3673/3673 [00:55<00:00, 66.41it/s]


,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,prevgoals90_-5,bpassesrcvd90_-5,exits90_-5,gkaerduels90_-5,fk90_-5,directfk90_-5,directfkontrgtpct_-5,corners90_-5,pens_-5,pencnvrspct_-5
0,Douglas Costa,27,18/19,55.0,Germany,L1,Bayern München,Italy,IT1,Juventus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G. Bonaventura,30,18/19,55.0,Italy,IT1,Juventus,Italy,IT1,Milan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R. Nainggolan,30,18/19,45.0,Italy,IT1,Roma,Italy,IT1,Inter Milan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P. Schick,22,18/19,20.0,Italy,IT1,Sampdoria,Italy,IT1,Roma,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,L. Goretzka,23,18/19,40.0,Germany,L1,Schalke 04,Germany,L1,Bayern München,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668,N. Geertsen,23,23/24,0.7,Netherlands,NL1,Vitesse,Denmark,DK1,Lyngby,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3669,S. Hansen,27,23/24,0.65,Netherlands,NL1,Go Ahead Eagles,Netherlands,NL1,NEC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3670,E. Bekiroğlu,34,23/24,0.6,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,Ankaragücü,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3671,S. Bryzgalov,30,23/24,0.6,Russia,RU1,Sochi,Russia,RU1,Fakel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
columns_ordered = df.drop(columns=['market_value']).columns.to_list()[:17]
for i in [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4]:
    columns_ordered += sorted(df.columns[df.columns.str.contains(f'_{i}')].to_list())
df = df[columns_ordered]
df

,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,...,touchesbox90_post_4,weight_post_4,xa90_post_4,xa_post_4,xg90_post_4,xg_post_4,xga90_post_4,xga_post_4,ycards90_post_4,ycards_post_4
0,Douglas Costa,27,18/19,Germany,L1,Bayern München,Italy,IT1,Juventus,summer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G. Bonaventura,30,18/19,Italy,IT1,Juventus,Italy,IT1,Milan,summer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R. Nainggolan,30,18/19,Italy,IT1,Roma,Italy,IT1,Inter Milan,summer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P. Schick,22,18/19,Italy,IT1,Sampdoria,Italy,IT1,Roma,summer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,L. Goretzka,23,18/19,Germany,L1,Schalke 04,Germany,L1,Bayern München,summer,...,1.94,82.0,0.09,1.87,0.23,4.57,0.0,0.0,0.25,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668,N. Geertsen,23,23/24,Netherlands,NL1,Vitesse,Denmark,DK1,Lyngby,winter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3669,S. Hansen,27,23/24,Netherlands,NL1,Go Ahead Eagles,Netherlands,NL1,NEC,winter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3670,E. Bekiroğlu,34,23/24,Türkiye,TR1,Adana Demirspor,Türkiye,TR1,Ankaragücü,winter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3671,S. Bryzgalov,30,23/24,Russia,RU1,Sochi,Russia,RU1,Fakel,winter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
df.columns = df.columns.str.replace('_post', '')

In [106]:
df.to_csv('final.csv', index=False)